In [55]:
import pickle
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import scipy.stats as st

INPUT_FILE = 'pickle/df_arrivals_aggregated.pkl'

In [56]:
# Time parameters
STEP_IN_SECONDS = 30
CUSTOMER_SPEED_PER_STEP = 0.5
NUMBER_OF_HOURS = 8
number_of_steps_in_hour = lambda step_in_seconds: 60 * 60 / step_in_seconds
steps_in_hour = int(number_of_steps_in_hour(STEP_IN_SECONDS))

In [57]:
def read_input_customer_distribution(input_file=INPUT_FILE):
    with open(input_file, 'rb') as f:
        df = pickle.load(f)
    return df

In [58]:
def generate_customers_dist(df, sigma, steps_in_hour):
    avg_customer_per_step_list = []
    bias_aggregated = []

    for customers_in_hour in df["value"]:
        avg_customer_per_step = customers_in_hour / steps_in_hour
        avg_customer_per_step_list.append(avg_customer_per_step)
        # Generate samples
        mu = avg_customer_per_step
        bias_customers_in_hour = np.random.normal(mu, sigma, steps_in_hour)
        bias_customers_in_hour = [round(abs(x)) for x in bias_customers_in_hour]
        bias_aggregated.append(sum(bias_customers_in_hour))

    return bias_aggregated

In [59]:
def generate_multiple_customers_dist(df, runs, sigma, steps_in_hour):
    df_step_list = []

    for run in range(runs):
        df_step = pd.DataFrame()
        sample_customer = generate_customers_dist(df, sigma, steps_in_hour)
        df_step["value"] = sample_customer
        df_step["run"] = run
        df_step["hour"] = [i + 8 for i in range(len(sample_customer))]
        df_step_list.append(df_step)
    df_step_out = pd.concat(df_step_list)
    return df_step_out

In [60]:
def compute_confidence_interval(df_step_out):
    l_list, u_list = [], []
    for hour in df["hour"]:
        subset_df = df_step_out.query(f"hour=={hour}")
        values = subset_df["value"]
        l, u = st.t.interval(0.95, len(values)-1, loc=np.mean(values), scale=st.sem(values))
        l_list.append(l)
        u_list.append(u)
    return l_list, u_list

In [76]:
def aggregate_runs(df_step_out, df):
    df_step_avg = df_step_out.groupby(by=["hour"]).mean()
    df_step_avg = df_step_avg.reset_index()
    # Confidence interval
    l_list, u_list = compute_confidence_interval(df_step_out)
    # Final dataframe
    df_full = pd.DataFrame()
    df_full["hour"] = df_step_avg["hour"]
    df_full["step_avg"] = df_step_avg["value"]
    df_full["real"] = df["value"]
    df_full["lower"] = l_list
    df_full["upper"] = u_list
    df_full["error"] = abs(df_full["step_avg"] - df_full["real"]) / df_full["real"]
    return df_full


In [85]:
# Create traces
def plot_distributions(df_full):

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_full["hour"], y=df_full["real"],
                        mode='lines',
                        name='True distribution'))
    fig.add_trace(
        go.Scatter(
            x=df_full["hour"],
            y=df_full["step_avg"],
            mode='lines',
            name='Synthetic step distribution',
            error_y=dict(
                type='data',
                symmetric=False,
                array=np.array(df_full["upper"]) - np.array(df_full["step_avg"]),
                arrayminus=np.array(df_full["step_avg"]) - np.array(df_full["lower"]))
            ))
    fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 1
        ),
        yaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 15
        ),
        title={
            'text': '<br>'.join(["Real customer distribution", "vs",
                                 "Synthetic aggregated per-step biased distribution"]),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title="Hour",
        yaxis_title="Number of incoming customers",
        legend_title="Distribution",
    )
    return fig

In [86]:
sigma = 0.4
n_runs = 30

df = read_input_customer_distribution()
df_step_out = generate_multiple_customers_dist(df, n_runs, sigma, steps_in_hour)
df_full = aggregate_runs(df_step_out, df)
fig = plot_distributions(df_full)
fig.show()

In [103]:
def plot_distribution_error(df_full):

    fig = go.Figure()
    fig.add_trace(go.Bar(x=df_full["hour"], y=df_full["error"]))
    fig.update_layout(
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 1
        ),
        yaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 0.1
        ),
        title={
            'text': 'Relative error for customer distribution',
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        xaxis_title="Hour",
        yaxis_title="Relative error",
        legend_title="Store",
    )
    return fig

In [104]:
fig = plot_distribution_error(df_full)
fig.show()

